<a href="https://www.kaggle.com/code/member09/toxiccommentclassifierbert?scriptVersionId=187344275" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/toxic-data-output/submission3.csv


In [3]:
train_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")

sample_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip")
test_labels_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip")
test_df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")


print(test_df.shape)
test_df.head()

(153164, 2)


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [4]:
print(sample_df.shape)
sample_df.head()

(153164, 7)


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [5]:
print(test_labels_df.shape)
test_labels_df.head()

(153164, 7)


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [6]:
print(train_df.shape)
train_df.head()


(159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train_df.isnull().sum()
print(train_df.shape)
train_df[(train_df["toxic"]==0) & (train_df["severe_toxic"] == 0) & (train_df["obscene"] == 0) & (train_df["threat"] == 0) & (train_df["insult"] == 0) & (train_df["identity_hate"] == 0) ]

train_df["is_hate_comment"] = (train_df["toxic"]>0) | (train_df["severe_toxic"]>0) | (train_df["obscene"]>0) | (train_df["threat"]>0) | (train_df["insult"]>0) | (train_df["identity_hate"]>0)
train_df

print(f"Total non-toxic comments : {train_df[train_df['is_hate_comment']==False].shape[0]}")
print(f"Total toxic comments : {train_df[train_df['is_hate_comment']==True].shape[0]}")

(159571, 8)
Total non-toxic comments : 143346
Total toxic comments : 16225


In [8]:
train_df.sort_values(by = ['toxic'], ascending = False).head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate_comment
83844,e061b485cfa88d4e,Vandalism of my user page \n\nPlease do not va...,1,0,1,0,1,0,True
145225,190748c37c8dc159,"Douche bag, know the band Sublime before you t...",1,0,1,0,1,0,True
54826,9284484fe7d8fdd9,Interesting fact \n\nYou're a huge fucking too...,1,0,1,0,1,0,True
35554,5f0120e927cf78d9,"""::What point do you think you have? There's w...",1,0,0,0,0,0,True
145216,18f35bf40d79d277,FUCK YOU!!!! stupid IGNORANT!!!!! \n\nHIM\nis ...,1,0,1,0,0,0,True
145220,18fd5b4427dd0b33,Why are you being such a nigger lover/faggot a...,1,0,1,0,1,0,True
15868,29e73351002d063c,THE LINK IS THERE. I have an idea. Why not del...,1,0,1,0,1,0,True
95543,ff7cd88f0588f00d,"Shit I hate you, you pompous bastard. 110.32.1...",1,0,1,0,1,0,True
95542,ff7ab6da3cce3b07,"fisrt, ANYONE WHO BELEIVES THIS CRAP IS A FOOL",1,0,1,0,0,0,True
127494,a9df5f43821cb576,I'm the bigot? you are a pathetic racist Afroc...,1,0,0,0,1,1,True


In [9]:
# how manys
print(f"Toxic : {train_df[train_df['toxic']==1].shape[0]}")
print(f"severe_toxic : {train_df[train_df['severe_toxic']==1].shape[0]}")
print(f"obscene : {train_df[train_df['obscene']==1].shape[0]}")
print(f"threat : {train_df[train_df['threat']==1].shape[0]}")
print(f"insult : {train_df[train_df['insult']==1].shape[0]}")
print(f"identity_hate : {train_df[train_df['identity_hate']==1].shape[0]}")

Toxic : 15294
severe_toxic : 1595
obscene : 8449
threat : 478
insult : 7877
identity_hate : 1405


In [10]:
train_neg_df = train_df[train_df["is_hate_comment"]==True]
train_pos_df = train_df[train_df["is_hate_comment"]==False]

train_neg_df.shape, train_pos_df.shape

((16225, 9), (143346, 9))

In [11]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

In [12]:
train_text = train_df["comment_text"]
train_labels = train_df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]]
train_labels

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [13]:
# clean text
import re
from string import punctuation
def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
#     text = re.sub(r"<a[^>]*>(.*?)</a>", r"\1", text) # remove anchor tags
#     text = re.sub(r"<.*?>", " ", text) # remove html tags
#     text = re.sub(r"[^A-Za-z0-9\s]+", "", text) # special characters
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    return text

train_df["comment_text"] = train_df["comment_text"].map(preprocess_text)

In [14]:
train_df["comment_text"][:10]

0    explanation why the edits made under my userna...
1    daww he matches this background colour im seem...
2    hey man im really not trying to edit war its j...
3    more i cant make any real suggestions on impro...
4    you sir are my hero any chance you remember wh...
5    congratulations from me as well use the tools ...
6         cocksucker before you piss around on my work
7    your vandalism to the matt shirvington article...
8    sorry if the word nonsense was offensive to yo...
9    alignment on this subject and which are contra...
Name: comment_text, dtype: object

In [15]:
# , torch_dtype=torch.float16, attn_implementation="sdpa"

In [16]:

class BERTDataset:
    def __init__(self,df, texts, labels, max_len=128):
        self.df = df
        self.texts = texts
        # for multi class
        self.labels = labels #df[labels].values
        self.max_len = max_len
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.num_examples = len(self.texts)
    
    def __len__(self):
        return self.num_examples

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        tokenized_text = self.tokenizer(text, add_special_tokens=True, padding="max_length", max_length=self.max_len, truncation=True)
        ids = tokenized_text["input_ids"]
        mask = tokenized_text["attention_mask"]
        token_type_ids = tokenized_text["token_type_ids"]
        return {"ids":torch.tensor(ids, dtype=torch.long),
                "mask": torch.tensor(mask, dtype=torch.long),
                "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                "target" : torch.tensor(label, dtype=torch.float)
                }


In [17]:
train_df.shape[0], train_df.shape[0] // 5

df_val = train_df[:30000]
df_train = train_df[30000:]

df_val.shape, df_train.shape

((30000, 9), (129571, 9))

In [18]:
# train_ds = BERTDataset(df_train, df_train["comment_text"].values, df_train["is_hate_comment"].values)
# val_ds = BERTDataset(df_val, df_val["comment_text"].values, df_val["is_hate_comment"].values)
# df_train = df_train[:1000]
# df_val = df_val[:1000]
train_ds = BERTDataset(df_train, df_train["comment_text"].values, df_train[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].values)
val_ds = BERTDataset(df_val, df_val["comment_text"].values, df_val[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].values)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [19]:
len(train_ds), len(val_ds)
# train_ds[0]["target"]

(129571, 30000)

In [20]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=32,drop_last=True, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=32, drop_last=True, shuffle=True)

In [22]:
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,is_hate_comment
30000,4fa70cdc36daf238,pan american union final act of the second mee...,0,0,0,0,0,0,False
30001,4fa7e0bb90941cf5,wpcivil be cautious even when we disagree we m...,0,0,0,0,0,0,False
30002,4fa8efded29d957d,mamas family wasnt a revert changed the title ...,0,0,0,0,0,0,False
30003,4fa99960927b1d90,european court of human rights echr decision o...,0,0,0,0,0,0,False
30004,4faa27cefc11ee55,heads up see this ani thread about your old us...,0,0,0,0,0,0,False


In [1]:

class ToxicModel(nn.Module):
    def __init__(self, num_classes):
        super(ToxicModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased", return_dict=False)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_classes)


    def forward(self, input_ids, token_type_ids, attention_mask, labels=None):
        _, pooled_output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits


def train(model, train_dataset, valid_dataset, epochs=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for batch in train_dataset:
            optimizer.zero_grad()
            ids = batch["ids"].to(device)
            mask = batch["mask"].to(device)
            token_ids = batch["token_type_ids"].to(device)
            target = batch["target"].to(device)
#             print(mask.shape, token_ids.shape)
            logits = model(ids, attention_mask=mask, token_type_ids=token_ids)
            loss = criterion(logits, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * batch["ids"].size(0)
        train_loss = train_loss / len(train_dataset)
        model.eval()
        valid_loss = 0

        for batch in valid_dataset:
            ids = batch["ids"].to(device)
            mask = batch["mask"].to(device)
            token_id = batch["token_type_ids"].to(device)
            target = batch["target"].to(device)
            logits = model(ids, attention_mask=mask, token_type_ids=token_ids)
#             print(logits.shape, target.shape)
            loss = criterion(logits, target)
    #         loss = logits.loss
            valid_loss += loss.item() * batch["ids"].size(0)
        valid_loss = valid_loss / len(valid_dataset)

        print(
          f"Epoch : {epoch+1} / {epochs} ..",
          f"Train loss : {train_loss:.3f} ",
          f"Validation loss : {valid_loss:.3f} "
        )



NameError: name 'nn' is not defined

In [ ]:
model = ToxicModel(num_classes=6)
train(model, train_dataloader, valid_dataloader, epochs=5)

In [ ]:
len(train_dataloader)

In [ ]:
# output_model = '/toxicmodel.pth'
# torch.save({'model_state_dict': model.state_dict()}, output_model)


In [ ]:

# # load
# checkpoint = torch.load(output_model)
# model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:

# torch.save(model, '/toxicmodel2.pth')
# 


In [ ]:
# torch.save(train_ds.tokenizer, '/toxicmodeltokenizer2.pth')



In [ ]:
# import torch
# saved_model = torch.load('/toxicmodel2.pth')
# saved_tokenizer = torch.load('/toxicmodeltokenizer2.pth')

In [ ]:
# saved_tokenizer

In [ ]:
# saved_model

In [ ]:
# sample_df

test_df["comment_text"] = test_df["comment_text"].map(preprocess_text)
test_df
# tokenized_text = saved_tokenizer(test_df["comment_text"], add_special_tokens=True, padding="max_length", max_length=128, truncation=True)

## Test data

In [ ]:
test_df

In [ ]:
test_df["toxic"]=pd.Series(dtype='float')
test_df["severe_toxic"] = pd.Series(dtype='float')
test_df["obscene"] = pd.Series(dtype='float')
test_df["threat"] = pd.Series(dtype='float')
test_df["insult"] = pd.Series(dtype='float')
test_df["identity_hate"] = pd.Series(dtype='float')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# test_df.to(device)
for idx, row in test_df.iterrows():
    idx = int(idx)
    comment = str(row["comment_text"])
    batch = saved_tokenizer(comment, add_special_tokens=True, padding="max_length", max_length=128, truncation=True,return_tensors='pt')
    
    ids = batch["input_ids"].to(device)
    mask = batch["attention_mask"].to(device)
    token_ids = batch["token_type_ids"].to(device)
    logits = saved_model(ids, attention_mask=mask, token_type_ids=token_ids)
    # logits to probabilities
    predicted_probs = torch.nn.functional.softmax(logits, dim=1)
#     print(type(idx), len(logits[0].detach().cpu().numpy()))
    print(predicted_probs)
    test_df.iloc[idx,-6:] =predicted_probs.detach().cpu().numpy()

In [ ]:
test_df.to_csv("submission3.csv")

In [6]:
import pandas as pd
df = pd.read_csv("/kaggle/input/toxic-data-output/submission3.csv")
df[["id","toxic","severe_toxic","obscene","threat","insult","identity_hate"]].to_csv("final_submission.csv", index=False)

In [7]:
dff = pd.read_csv("final_submission.csv")
dff

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.740694,0.000251,0.221042,0.000003,0.037922,8.882750e-05
1,0000247867823ef7,0.435073,0.124694,0.169969,0.061909,0.148863,5.949291e-02
2,00013b17ad220c46,0.349234,0.161542,0.195021,0.079283,0.144081,7.083908e-02
3,00017563c3f7919a,0.342428,0.161851,0.188166,0.088774,0.149359,6.942125e-02
4,00017695ad8997eb,0.386070,0.149724,0.169953,0.085069,0.147026,6.215877e-02
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.478965,0.115410,0.149502,0.056365,0.149130,5.062808e-02
153160,fffd7a9a6eb32c16,0.403873,0.141588,0.167575,0.083121,0.140924,6.291729e-02
153161,fffda9e8d6fafa9e,0.310636,0.176400,0.207750,0.089858,0.141426,7.392969e-02
153162,fffe8f1340a79fc2,0.299259,0.181533,0.203792,0.098413,0.140052,7.695119e-02


/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
